Next steps:

1. Format data into array of pixel vectors:

|Pixel|Vector|(x,y) coordinate|
|---|---|---|
|0|[Al-value, Fe-value, ..., Ce-value]|(0,0)|
|1| ... | (0,1)|
|...|...|...|

2. Format the masks into an array of binary values:

|Pixel|Yes/No|(x,y) coordinate|
|---|---|---|
|0  |0  |(0,0)|
| 1 | 0 | (0,1)|
|...|...|...|
| n | 1 | (x,y)|

3. Multiply the data array by the mask array (to make every black mask pixel 0 in the data). Remove all vectors that are now [0,0,0,0,...,0] as a result.
4. Repeat for each mineral phase. 

4. Do a PCA test on each mineral phase with the array from step 3. Analyze the results and tune the test as needed using the PCA documentation (MLGeo book).

5. Construct histograms where the x-axis is an element (with bins for each possible value) and the y axis is the observed count. Make confidence intervals on the histograms to determine the possible range of values of an element in a specific mineral phase. 

6. Use the value ranges of the main 3 primary component elements to generate new masks (by filtering the 2D arrays of those element channels for the required value range), then plug them into our image compiler to create an RGB image of ONLY pixels identified as that element. 
 =========== =========== =========== =========== =========== =========== ===========

*Data characterization*
How unbalanced is our dataset? How many of each class are available in our dataset?

# Create a data structure


|Index #| Pixel_Coordinates | Feature_Vector            | Label           | 
|--|-------------------:|---------------------------|-----------------|
|0| (x1, y1, z1)       | [feature_vector1]         | [label1]        |
|1| (x2, y2, z2)       | [feature_vector2]         | [label2]        |
|...| ...               | ...                       | ...             |
|n|(x,y,z)               | [ElementA-value, elementB-value]         | Garnet            |



For each thin section, we first load all maps and masks into a list, using the filenames as keys and the numpy arrays of the data as the values. 
Next, we want to extract the element names and mineral names from the map and mask filenames. 
Then, we want to extract the pixel coordinates and add them to the list. 
We then reshape the data such that each pixel has its own row and all the map data is stored in a *feature vector* for each pixel. 

This dataset is then stored as a Pandas DataFrame. 

## Reshaping for PCA test.
We will do a PCA test on a single thin section, for which we have all masks. For each mineral phase, we have make a copy of the feature vector data and multiply it by the binary mask values for that mineral. Then, we remove all pixels that have all 0's in the feature vector. We save this new data series as a new column. This is the PCA-ready data. There should be one such column for each mineral phase. 


The dataset is created as a pandas DataFrame for each thin section, which can then be combined into an xarray. 
|Pixel|coordinates|Feature vectors|Garnet mask|Staurolite mask|Quartz mask| ...|
|---|---|---|---|---|---|---|
|Index #s| (x,y)|n-D array|1 or 0|1 or 0|1 or 0|---|


Import packages.

In [23]:
import pandas as pd
import numpy as np
import h5py
import os
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

# Formatting RAW maps for HPF5
Load tiffs and filenames in, process, and insert into a thin section dataframe. 

In [24]:
# let's first get lists of all the thin sections, the channels, and the masks we'll need to access:

datafolder = './Aikin_Data'

# create an HDF5 file to store the dataframes into in the loop
datafile = h5py.File(datafolder + '/Aikin_ML_data.h5', 'w') # the 'w' is for 'writing'
datafile.create_group('RAW_maps')

# create a sorted list of all folder names in the data folder except '.DS_Store/'
tsNames = sorted([entry for entry in os.listdir(datafolder) 
                if os.path.isdir(os.path.join(datafolder, entry)) and 'DS_Store' not in entry])

# initialize a list as an additional way to access the data for easy processing within the notebook
RAW_tsList = []


# now we can use the names list to open the files and the stacks list to store them. 
# for each thin section folder:
for ts in range(len(tsNames)):
    
    # get the name of the thin section
    this_tsName = tsNames[ts]
    # initialize a dict for a dataframe
    this_ts_data = {}

    # list all the filenames of maps (in RAW/ folder) within the thin section folder
    mapNames = sorted([entry for entry in os.listdir(os.path.join(datafolder, this_tsName, 'RAW'))
                if 'DS_Store' not in entry])

    ### we'll want to clean up the filenames for the dataset labels.
    # What are the element names for each map?
    # apply text extraction to the mapNames to get the element names. 
    elementNames = []
    for mapName in mapNames:
        text = mapName
        # list the prefixes used in the raw data files in the same order as mapNames
        filterList = 'UGG-W3-87.7-10.1-Full_', 'UGG-W3-78.7-10-',  'CC-84.7-R21-NA2-1_full_', 'CC-84.7-NA2.2_'
        filter_str = filterList[ts]
        index = text.find(filter_str)
        extracted_str = text[index + len(filter_str): index + len(filter_str) + 2]
        elementNames = np.append(elementNames, extracted_str)


    # iteration counter serves to allow coordinate vectors to be generated during the first iteration only.if iteration 
    iteration = 0
    # for each map, load the file, turn it into an array, and append it to the data dict object.
    for map in range(len(mapNames)):
        # open the map
        filename = mapNames[map]
        rootfolder = os.path.join('./Aikin_Data', this_tsName, 'RAW')
        single_tsMap = Image.open(os.path.join(rootfolder, filename))
        # as numpy array
        single_mapArray = np.asarray(single_tsMap)
        # add a coordinates vector list (x,y,z=channel)
        if iteration == 0:
            x_coords = []
            y_coords = []
            x, y = single_mapArray.shape
            for i in range(x):
                for j in range(y):
                        coords = [i,j]
                        x_coords.append(coords[0])
                        y_coords.append(coords[1])

        x_coords = np.array(x_coords, dtype=np.float32)
        y_coords = np.array(y_coords, dtype=np.float32)

        this_ts_data['x_coords'] = x_coords
        this_ts_data['y_coords'] = y_coords

        # and flattened
        flattened_mapArray = np.ndarray.flatten(single_mapArray)
        columnName = str(elementNames[map]) + ' map'
        this_ts_data[columnName] = flattened_mapArray
        iteration += 1

    
    # create a pandas DataFrame out of the thin section data dict object 
    this_ts_df = pd.DataFrame(this_ts_data)
    RAW_tsList.append(this_ts_data)
    # add the dataframe to the hpf5 file as a new group
    datafile['RAW_maps'].create_dataset(this_tsName, data=this_ts_df)

datafile.close()

# Creating the masks dataframes



In [129]:
# next, we'll extract the raw data from our HDF5 file, then create a copy of it and multiply the copy by the corresponding mask.
# which masks correspond to which maps? it's thin section by thin section, so the output should be a 9-channel array for each mineral type. 
# where is the data file?
filepath = '/Aikin_ML_data.h5'
# what is the name of the input dataset group?
input_group = 'RAW'

# store the datasets and their shapes in lists. 
input_datasets = []
input_datasetShapes = []

# Open the HDF5 file in read-only mode
with h5py.File(datafolder + filepath, 'r') as file:
        
    # Access the input datasets
    allMaps = file['RAW_maps']
    print('keys: ', allMaps.keys())
    for ts in allMaps:
        this_tsStack = allMaps[ts]
        # store the datasets in a list
        print('a: ', this_tsStack)
        input_datasets.append(this_tsStack)
        # this_tsStackArray = np.asarray(this_tsStack, dtype=int)
        # print(this_tsStackArray)
        # store the shapes of datasets in a list. 
        rows, cols = this_tsStack.shape
        input_datasetShapes.append((rows,cols))



    # for each thin section,
    for ts in range(len(input_datasets)):
        # make a copy of the input dataset (ts/RAW/)
        this_tsMaps = np.copy(input_datasets[ts])
        print('this_tsMaps:', this_tsMaps.shape)
        # get the masks for this ts (ts/masks/)
        this_tsMasks = allMasks_list[ts]
        # list their names separately
        this_maskNames = list(this_tsMasks.keys())
        # for each mask,
        for mask in range(len(this_tsMasks)):
            single_maskedMapStack = {}
            # get the one mask at a time (the dict is indexed with key 'strings')
            this_mask = this_tsMasks[this_maskNames[mask]]
            # for each column in this stack of maps,
            for i in range(cols):
                # multiply the column by the mask
                single_masked_map = this_tsMaps[:, i] * this_mask[:]
                print(single_masked_map.shape)
                print('number of zeros in maskedmaps: ', np.count_nonzero(masked_maps))
                print('number of zeros in thismask: ', np.count_nonzero(this_mask))
                print('number of zeros in this_tsmap: ', np.count_nonzero(this_tsMaps[:,i]))
                print('number of zeros in diff:', np.count_nonzero(masked_maps) - np.count_nonzero(this_mask))
                
                #element = RAW_tsList[ts].keys()
                print(RAW_tsList[ts].keys())
                mineral = this_maskNames[mask]

                columnName =  mineral + '-mask_' + element
                single_maskedMapStack[columnName] = single_masked_map
                print(columnName)
                print(single_maskedMapStack)
            #columnName = {mineral_mask_element}
            #this_ts_maskedMaps[columnName] = 
# next the output needs to be checked and added to the hdf5 file in a new group. 
#     # Create or overwrite the output dataset in the same file
#     x = file.create_group('masked_maps')
#     output_dataset = file.create_dataset(dataset_name + '_modified', data=temp_dataset)


# #Create a new HDF5 file in write mode
# with h5py.File(filepath, 'a') as file:  # Open the same file in append mode


keys:  <KeysViewHDF5 ['78.7-10-1_Hot', '78.7-10-2_Hot', '84.7-NA-2-1_Cold', '84.7-NA2-2_Cold']>
a:  <HDF5 dataset "78.7-10-1_Hot": shape (800640, 12), type "<f4">
a:  <HDF5 dataset "78.7-10-2_Hot": shape (800640, 12), type "<f4">
a:  <HDF5 dataset "84.7-NA-2-1_Cold": shape (773300, 11), type "<f4">
a:  <HDF5 dataset "84.7-NA2-2_Cold": shape (589950, 11), type "<f4">
this_tsMaps: (800640, 12)
(800640,)
number of zeros in maskedmaps:  193615
number of zeros in thismask:  27501
number of zeros in this_tsmap:  799488
number of zeros in diff: 166114
dict_keys(['x_coords', 'y_coords', 'Al map', 'Ca map', 'Ce map', 'Fe map', 'K  map', 'Mg map', 'Ti map', 'Y  map', 'Zr map', 'll map'])


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U12'), dtype('<f4')) -> None

In [135]:
# use this to make the hdf5 datasets nicely labelled. need to work out the groups as inbetween step
# Create some data
data1  = np.arange(100.)
data2  = 2.0*data1
data3  = 3.0*data1
data4  = 3.0*data1

# use namesList to define dtype for recarray
namesList = ['height', 'mass', 'velocity', 'gravity']
ds_dt = np.dtype({'names':namesList,'formats':[(float)]*4 }) 

rec_arr = np.rec.fromarrays([data1, data2, data3, data4], dtype=ds_dt)
#print(rec_arr)
with h5py.File("experimentReadings.hdf5", "w") as h5f :

    dset = h5f.create_dataset("physics", (100,), data=rec_arr)
    print(dset['mass'])

[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.  20.  22.  24.  26.
  28.  30.  32.  34.  36.  38.  40.  42.  44.  46.  48.  50.  52.  54.
  56.  58.  60.  62.  64.  66.  68.  70.  72.  74.  76.  78.  80.  82.
  84.  86.  88.  90.  92.  94.  96.  98. 100. 102. 104. 106. 108. 110.
 112. 114. 116. 118. 120. 122. 124. 126. 128. 130. 132. 134. 136. 138.
 140. 142. 144. 146. 148. 150. 152. 154. 156. 158. 160. 162. 164. 166.
 168. 170. 172. 174. 176. 178. 180. 182. 184. 186. 188. 190. 192. 194.
 196. 198.]


# Opening the files



In [40]:
import h5py

datafile = h5py.File('./Aikin_data/Aikin_ML_data.h5', 'r')

# What is stored in this file? h5py.File acts like a Python dictionary, thus we can check the keys,

list(datafile.keys())

# examine a data set as a Dataset object. dset is not an array but an HDF5 dataset.
this_ts = datafile['RAW_maps']
list(this_ts.keys())
dset = this_ts[list(this_ts.keys())[0]]
dset.shape
dset.dtype

#we can use indexing to read and write from dset. 

dtype('<f4')

## MLGeo-Book PCA test

In [ ]:
# PCA

# Import useful modules
import requests, zipfile, io, gzip, glob, os
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as ln
import pandas as pd
import matplotlib
from matplotlib import cm
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn import datasets
import sklearn



In [ ]:
# Feature selection via parameter exploration

# Heatmap functions
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar

def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
# Let's load in the iris dataset
iris = datasets.load_iris()

# Convert iris to a pandas dataframe...
irisDF = pd.DataFrame(data=iris.data,  
                  columns=iris.feature_names)

# Now, plot up sepal length vs width, color-coded by target (or species)

scatter = plt.scatter(irisDF['sepal length (cm)'], irisDF['sepal width (cm)'], c=iris.target)
plt.xlabel('sepal length (cm)')
plt.ylabel('sepal width (cm)')
plt.legend(scatter.legend_elements()[0], iris.target_names, title="Classes")


In [ ]:
# Now, how might we reduce these dimensions? 
# One way is by looking at how variables are correlated
# Calculate the correlation coefficients for all variables
allCorr = irisDF.corr()

im, _ = heatmap(allCorr, irisDF, irisDF,
                cmap="PuOr", vmin=-1, vmax=1,
                cbarlabel="correlation coeff.")

annotate_heatmap(im, size=7)

plt.tight_layout()
plt.show()


In [ ]:
allCorr

# PCA Test 
based on [Principal Component Analysis For Image Data in Python](https://www.askpython.com/python/examples/principal-component-analysis-for-image-data)





In [ ]:
# 

# show first 3 PCs
n_components = 3
pandas.DataFrame(pca.transform(df), columns=['PCA%i' % i for i in range(n_components)], index=df.index)

## File format

We are using [**HDF5**]() to store our raw and processed data, because it is well-integrated with python, works well and efficiently for large, numerical data, and allows us to store several pandas DataFrames in one file with labels, making it easy to access and recast the data. With this in mind, the data is divided by thin section, and subdivided into DataFrames in the same hierarchy as the `./data/` directory, with additional labeled DataFrames added for processed data. 

|**Data type** |description|dimensions|
|---|---|---|
|RAW/|...|...|
| masks/|...|...|
|...|...|...|


In [ ]:
import h5py

# Create an HDF5 file
with h5py.File('Aikin_ML_data', 'w') as hf: # the 'w' is for 'writing'
    # Store each DataFrame as a separate group
    for df_name, df in your_dataframes.items():
        hf.create_group(df_name)
        hf[df_name].create_dataset('data', data=df)

TypeError: Object dtype dtype('O') has no native HDF5 equivalent

In [11]:

this_ts_df['coordinates']

0              (0, 0)
1              (0, 1)
2              (0, 2)
3              (0, 3)
4              (0, 4)
             ...     
800635    (694, 1147)
800636    (694, 1148)
800637    (694, 1149)
800638    (694, 1150)
800639    (694, 1151)
Name: coordinates, Length: 800640, dtype: object

In [9]:
import h5py
import pandas as pd
import numpy as np

# Create a sample DataFrame with tuples
data = {'Column1': [(1, 2), (3, 4), (5, 6)],
        'Column2': [(7, 8), (9, 10), (11, 12)]}

df = pd.DataFrame(data)

# Create an HDF5 file and store the DataFrame
with h5py.File('data.h5', 'w') as hf:
    hf.create_dataset('my_dataframe', data=df)

TypeError: Object dtype dtype('O') has no native HDF5 equivalent